In [1]:
using DifferentialEquations
using LinearAlgebra
using Plots
using FileIO, JLD2

In [2]:
include("solvers.jl")
include("plot_functions.jl")
include("useful_functions.jl")
include("point_generators.jl")

points_on_sphere (generic function with 2 methods)

Introduces the same variable values as in "pipcorrelationvariables.ipynb". We need these to make good plots.

In [ ]:
Δt = 0.1
T = 60
N = Integer(T/Δt)

Loads the solution matrix created in "pipcorrelationvariables.ipynb" which we want to use as our "true" solution into the workspace

In [4]:
Mtrue = FileIO.load(raw"variables\RK4\640.jld2","M");

Chose what solvers and what timesteps we want to use for the plots. The "steps_vec" variable represent the different timesteps and should only contain numbers used in the "steps_vec" variable in "pipcorrelationmatrix.ipynb" when creating the solutionmatrices which we'll use here

In [21]:
solvers = [RK4]
steps_vec = [10 20 40 80 160 320 640]


Creates and saves heatmaps of the correlation for different solvers and timesteps with the "true" solution, depending on time and initial values.  $\newline$ 
Here we use the other solution matrices created in "pipcorrelationmatrix.ipynb".

In [ ]:
for solver = solvers
    for steps = steps_vec 
        
        loadString = string("variables", "\\", solver, "\\", steps, ".jld2")
        M = FileIO.load(loadString,"M")

        c = correlation_matrix(M, Mtrue)
        plot_pip_plot(c, Δt, clim=(0.7,1))
        
        saveString = string("plots", "\\", "Correlation", "\\", solver, "\\", steps, ".png")
        savefig(saveString)
    end
end

Creates and saves heatmaps of average correlation between chosen solvers and the "true" solution, depending on time and timestep size.

In [30]:
l = length(steps_vec)
x = [0:Δt:T]
y = [1:l]
for solver = solvers
    Mavg = zeros(l, N+1)
    for j = 1:l
        loadString = string("Variables", "\\", solver, "\\", steps_vec[j], ".jld2")
        M = FileIO.load(loadString,"M")
        c = correlation_matrix(M, Mtrue)
        Mavg[j,:] = sum(c,dims=1)/size(c)[1]
    end
    saveString = string("plots", "\\", "correlation", "\\", solver, "\\", solver,"_average", ".png")
    title = string(solver, " vs RadauIIA5 Δt=1E-5")
    heatmap(x,y,Mavg, ylabel="k", xlabel="time t", clim=(0.7,1), cmap=:grayC, title=title)
    savefig(saveString)
end